In [1]:
import os
from langchain.prompts import PromptTemplate
# from langchain_community.llms import Ollama
from langchain_ollama import OllamaLLM
from langchain.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Prompt

In [2]:
prompt_template = """
    You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate(template= prompt_template)
PROMPT

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. \n    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n    {context}\n    Question: {question}\n    Answer:")

# Model

In [3]:
llm = OllamaLLM(model="llama3.2")
llm

OllamaLLM(model='llama3.2')

In [4]:
# testing the llm
llm.invoke("Hello!")

'How can I assist you today?'

# Document Loaders

In [5]:
# loading documents
all_documents = []

files = ['./data/aboutme.txt', './data/cv.txt']
for file in files:
    loader = TextLoader(file)
    documents = loader.load()
    all_documents.extend(documents)

In [6]:
all_documents

[Document(metadata={'source': './data/aboutme.txt'}, page_content='My Personal Information:\n\nName: Bidhan Bajracharya\nAge: 23 years (as of 2025)\nBirthdate: September 14, 2001\nGender: Male\nNationality: Nepalese\nHeight: 5\'10"\nSkin tone: Brown\n\nEducation Background:\n- Master’s Degree: Data Science and Artificial Intelligence (DSAI)\n    - Institution: Asian Institute of Technology (AIT)\n\n- Bachelor’s Degree: BSc (Hons) in Computing\n    - Institution: London Metropolitan University\n\nProfessional Experience:\n- Software Engineer (1 year)\n    - Industry: Education\n- Currently not employed\n\nTechnical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn\n\nBeliefs & Philosophy:\n- Technology as a Double-Edged Sword:– If used ethically, technology can drive societal progress, but if misused, it can lead to negative consequences.\n- Data-Driven Decision Making:– Makin

## Document Transformers

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc = text_splitter.split_documents(all_documents)

In [8]:
doc 

[Document(metadata={'source': './data/aboutme.txt'}, page_content='My Personal Information:\n\nName: Bidhan Bajracharya\nAge: 23 years (as of 2025)\nBirthdate: September 14, 2001\nGender: Male\nNationality: Nepalese\nHeight: 5\'10"\nSkin tone: Brown\n\nEducation Background:\n- Master’s Degree: Data Science and Artificial Intelligence (DSAI)\n    - Institution: Asian Institute of Technology (AIT)\n\n- Bachelor’s Degree: BSc (Hons) in Computing\n    - Institution: London Metropolitan University\n\nProfessional Experience:\n- Software Engineer (1 year)\n    - Industry: Education\n- Currently not employed\n\nTechnical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy, Matplotlib, Scikit-Learn'),
 Document(metadata={'source': './data/aboutme.txt'}, page_content='Technical Skills:\n- Programming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\n- Frameworks & Libraries: React, Node.js, Pandas, NumPy,

## Text Embedding Models

In [9]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\Bidhan\anaconda3\envs\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Vector Stores

In [10]:
vector_path = './vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [11]:
db_file_name = 'personal_info'

# load the vector store if it exists
if os.path.exists(os.path.join(vector_path, db_file_name)):
        vectordb = FAISS.load_local(
            folder_path=os.path.join(vector_path, db_file_name),
            embeddings=embedding_model,
            allow_dangerous_deserialization=True # because I'm sure the file is safe
        )
else:
    vectordb = FAISS.from_documents(
        documents = doc,
        embedding = embedding_model
    )

    vectordb.save_local(
        folder_path = os.path.join(vector_path, db_file_name)
    )

## Retrievers

In [12]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [13]:
retriever.get_relevant_documents("What is your name?")

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_26544\3674800939.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is your name?")


[Document(id='36d2e3b5-2ff9-42bd-86c9-b2220cc83df3', metadata={'source': './data/aboutme.txt'}, page_content='My name is Bidhan Bajracharya, and I am from Nepal. I am 23 years old, born on September 14, 2001. I am male, with a height of 5\'10" and brown skin.\n\nCurrently, I am pursuing a Master’s in Data Science and Artificial Intelligence (DSAI) at the Asian Institute of Technology (AIT). Before this, I completed my Bachelor of Science (Hons) in Computing from London Metropolitan University.'),
 Document(id='defda409-233a-4a28-a33e-363ecdcbb99d', metadata={'source': './data/cv.txt'}, page_content='Level 9 Tech | Front-end Developer Intern (Aug 2022 – Dec 2022)\n    - Collaborated with a team of developers to build and maintain web applications.\n    - Utilized Git and GitHub to manage project workflows.\n    - Developed new features and improved existing functionalities to enhance user experience.'),
 Document(id='92edc082-e0da-4dda-8303-4155adbe14e2', metadata={'source': './data/cv.

# Memory

In [14]:
from langchain.memory import ChatMessageHistory

In [15]:
history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [16]:
# history.add_user_message('hi')
# history.add_ai_message('Hello! I\'m a personal chatbot who knows a lot about Bidhan.Go ahead and ask me anything about him.')
# history.add_user_message('How are you')
# history.add_ai_message('I\'m quite good. How about you?')

In [17]:
history

InMemoryChatMessageHistory(messages=[])

In [18]:
from langchain.memory import ConversationBufferMemory

In [19]:
# memory = ConversationBufferMemory(return_messages = True)
# memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
# memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
# memory.load_memory_variables({})

In [20]:
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer")
# memory = ConversationBufferMemory(input_key="question", output_key="answer")

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_26544\4142404075.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer")


In [21]:
# memory.save_context({'question':'hi'}, {'answer':'What\'s up?'})
# memory.save_context({"question":'From now on every reply, you can use the context provided. And answer as bidhan himself.'},{'answer': 'Okay I will now answer as Bidhan.'})

# Chain

In [22]:
from langchain.chains import RetrievalQA

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [24]:
result = qa_chain({"query": "What is my birthdate?"})
answer = result.get('result', '')

C:\Users\Bidhan\AppData\Local\Temp\ipykernel_26544\1298083337.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": "What is my birthdate?"})


In [25]:
sources = []
source_docs = result.get('source_documents', [])
for doc in source_docs:
    source = {
        "content": doc.page_content[:150] + "...",
        "source": doc.metadata.get('source', 'Unknown')
    }
    sources.append(source)

In [26]:
sources

[{'content': 'My name is Bidhan Bajracharya, and I am from Nepal. I am 23 years old, born on September 14, 2001. I am male, with a height of 5\'10" and brown skin.\n\n...',
  'source': './data/aboutme.txt'},
 {'content': 'NAME: Bidhan Bajracharya\n\nEDUCATION:\nAsian Institute Of Technology\nMaster of Science (M.S) - Data Science and AI | August 2024 - May 2026\n\nIslington C...',
  'source': './data/cv.txt'},
 {'content': '- Uddum (Vue.js, Nuxt.js)\n    - Developed an online airplane ticket booking platform.\n    - Worked with experienced developers to optimize and resolve...',
  'source': './data/cv.txt'}]

In [27]:
answer

'Your birthdate is September 14, 2001. You are currently 23 years old.'

In [28]:
questions = [
 "How old are you?",
 "What is your highest level of education?",
 "What major or field of study did you pursue during your education?",
 "How many years of work experience do you have?",
 "What type of work or industry have you been involved in?",
 "Can you describe your current role or job responsibilities?",
 "What are your core beliefs regarding the role of technology in shaping society?",
 "How do you think cultural values should influence technological advancements?",
 "As a master's student, what is the most challenging aspect of your studies so far?",
 "What specific research interests or academic goals do you hope to achieve during your time as a master's student?"
]

In [29]:
for idx, question in enumerate(questions, start=1):
    response = qa_chain({"query": question})
    print(f"{idx}) {question}")
    print(f"Answer: {response['result']}\n")

1) How old are you?
Answer: You mentioned that you were born on September 14, 2001. Therefore, based on the current year being not specified but typically 2023 or later, it can be assumed that as of 2023, you would be around 22 years old, considering a birth year in the 2000s. However, this exact calculation might vary depending on the current year and your birthdate, which is not provided.

2) What is your highest level of education?
Answer: Bidhan's highest level of education is a Master of Science (M.S) degree in Data Science and AI from the Asian Institute Of Technology, earned between August 2024 and May 2026.

3) What major or field of study did you pursue during your education?
Answer: I don't know. The context provided doesn't mention the student's major or field of study.

4) How many years of work experience do you have?
Answer: You don't know the answer, but according to your provided context, you mentioned having one year of experience as a Front-end Developer Intern from A

# Analysis

In [ ]:
# code here...

In [30]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [31]:
prompt_template = """
    You are an assistant that provides useful information about Bidhan. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    {context}
    Question: {input}
    Answer:
    """.strip()

PROMPT2 = PromptTemplate(template= prompt_template)

In [32]:
document_chain=create_stuff_documents_chain(llm, PROMPT2)

In [33]:
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [34]:
response=retrieval_chain.invoke({"input":"What is your highest level of education?"})

In [35]:
response

{'input': 'What is your highest level of education?',
 'context': [Document(id='92edc082-e0da-4dda-8303-4155adbe14e2', metadata={'source': './data/cv.txt'}, page_content='TECHNICAL SKILLS:\nProgramming Languages: Python, Java, JavaScript/TypeScript, C#, SQL\nTools: Git/GitHub, Jupyter Notebook, VS Code, IntelliJ, PyCharm, Docker, Tableau, AWS, PostgreSQL, MySQL\nLibraries & Frameworks: Pandas, NumPy, Matplotlib, Scikit-Learn, React.js'),
  Document(id='defda409-233a-4a28-a33e-363ecdcbb99d', metadata={'source': './data/cv.txt'}, page_content='Level 9 Tech | Front-end Developer Intern (Aug 2022 – Dec 2022)\n    - Collaborated with a team of developers to build and maintain web applications.\n    - Utilized Git and GitHub to manage project workflows.\n    - Developed new features and improved existing functionalities to enhance user experience.'),
  Document(id='9a542188-9885-4631-9ec8-4820d21ec060', metadata={'source': './data/cv.txt'}, page_content='NAME: Bidhan Bajracharya\n\nEDUCATION

In [36]:
for idx, question in enumerate(questions, start=1):
    response = retrieval_chain.invoke({"input": question})
    print(f"{idx}) {question}")
    print(f"Answer: {response['answer']}\n")

1) How old are you?
Answer: You mentioned that you are 23 years old, born on September 14, 2001.

2) What is your highest level of education?
Answer: Bidhan's highest level of education is a Master of Science (M.S) degree in Data Science and AI from the Asian Institute Of Technology, which he completed from August 2024 to May 2026.

3) What major or field of study did you pursue during your education?
Answer: I don't have information about the major or field of study pursued by Bidhan. The provided context only mentions Bidhan's academic roles, extracurricular activities, and technical skills. No details are available about their educational background.

4) How many years of work experience do you have?
Answer: I don't know the answer. You provided context about your one year of experience as a Front-end Developer Intern, but I'm not aware of any information that would allow me to calculate or confirm the total number of years of work experience you have now, considering you are curren